# Week 8 Lab session
 - Use logistic regression in predicting stock direction
 - Training/test split
 - logistic regression
 - LDA
 - QDA
 - k-NN
 - Analysis

In [1]:
import numpy as np
import pandas as pd

In [2]:
%ls *.csv

Advertising.csv* Caravan.csv*     Hitters.csv*     Khan_ytest.csv*
Auto.csv         Carseats.csv*    Khan_xtest.csv*  Khan_ytrain.csv*
Boston.csv*      Default.csv      Khan_xtrain.csv* Smarket.csv*


In [3]:
data = pd.read_csv('Smarket.csv')

In [4]:
data.head(10)

,Unnamed: 0,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
0,1,2001,0.381,-0.192,-2.624,-1.055,5.010,1.1913,0.959,Up
1,2,2001,0.959,0.381,-0.192,-2.624,-1.055,1.2965,1.032,Up
2,3,2001,1.032,0.959,0.381,-0.192,-2.624,1.4112,-0.623,Down
3,4,2001,-0.623,1.032,0.959,0.381,-0.192,1.2760,0.614,Up
4,5,2001,0.614,-0.623,1.032,0.959,0.381,1.2057,0.213,Up
5,6,2001,0.213,0.614,-0.623,1.032,0.959,1.3491,1.392,Up
6,7,2001,1.392,0.213,0.614,-0.623,1.032,1.4450,-0.403,Down
7,8,2001,-0.403,1.392,0.213,0.614,-0.623,1.4078,0.027,Up
8,9,2001,0.027,-0.403,1.392,0.213,0.614,1.1640,1.303,Up
9,10,2001,1.303,0.027,-0.403,1.392,0.213,1.2326,0.287,Up


In [5]:
data['direction2'] = data.Direction.factorize()[0]

In [6]:
data.head()

,Unnamed: 0,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction,direction2
0,1,2001,0.381,-0.192,-2.624,-1.055,5.010,1.1913,0.959,Up,0
1,2,2001,0.959,0.381,-0.192,-2.624,-1.055,1.2965,1.032,Up,0
2,3,2001,1.032,0.959,0.381,-0.192,-2.624,1.4112,-0.623,Down,1
3,4,2001,-0.623,1.032,0.959,0.381,-0.192,1.2760,0.614,Up,0
4,5,2001,0.614,-0.623,1.032,0.959,0.381,1.2057,0.213,Up,0


### Using the following pandas method, one can see how many positive (negative) cases are in a data set.

In [7]:
data.loc[data['direction2'] == 0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 648 entries, 0 to 1247
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  648 non-null    int64  
 1   Year        648 non-null    int64  
 2   Lag1        648 non-null    float64
 3   Lag2        648 non-null    float64
 4   Lag3        648 non-null    float64
 5   Lag4        648 non-null    float64
 6   Lag5        648 non-null    float64
 7   Volume      648 non-null    float64
 8   Today       648 non-null    float64
 9   Direction   648 non-null    object 
 10  direction2  648 non-null    int64  
dtypes: float64(7), int64(3), object(1)
memory usage: 60.8+ KB


In [8]:
data.loc[data['direction2'] == 1].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 602 entries, 2 to 1249
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  602 non-null    int64  
 1   Year        602 non-null    int64  
 2   Lag1        602 non-null    float64
 3   Lag2        602 non-null    float64
 4   Lag3        602 non-null    float64
 5   Lag4        602 non-null    float64
 6   Lag5        602 non-null    float64
 7   Volume      602 non-null    float64
 8   Today       602 non-null    float64
 9   Direction   602 non-null    object 
 10  direction2  602 non-null    int64  
dtypes: float64(7), int64(3), object(1)
memory usage: 56.4+ KB


### Import the logistic regression model from sklearn library

In [9]:
import sklearn.linear_model as skl_lm

In [10]:
x = data[['Lag1','Lag2','Lag3','Lag4','Lag5','Volume']].values
y = data['direction2'].values

In [11]:
clf_logit = skl_lm.LogisticRegression(solver='newton-cg')
clf_logit.fit(x,y)

LogisticRegression(solver='newton-cg')

In [12]:
clf_logit.coef_

array([[ 0.07284382,  0.04223527, -0.0110161 , -0.00928364, -0.01026345,
        -0.13210837]])

In [13]:
clf_logit.intercept_

array([0.12107884])

In [14]:
clf_logit.classes_

array([0, 1])

In [15]:
pred = clf_logit.predict(x)

In [16]:
from sklearn.metrics import confusion_matrix, classification_report

confusion_matrix(y, pred).T

array([[507, 458],
       [141, 144]])

In [17]:
print(classification_report(y, pred, digits=3))

              precision    recall  f1-score   support

           0      0.525     0.782     0.629       648
           1      0.505     0.239     0.325       602

    accuracy                          0.521      1250
   macro avg      0.515     0.511     0.477      1250
weighted avg      0.516     0.521     0.482      1250



In [18]:
(507+144)/(507+144+458+141)

0.5208

## Training and test subsets: data before and after 2005 

In [19]:
X_train = data.loc[data['Year']<2005][['Lag1','Lag2','Lag3','Lag4','Lag5','Volume']].values
y_train = data.loc[data['Year']<2005]['direction2'].values

X_test = data.loc[data['Year']>2004][['Lag1','Lag2','Lag3','Lag4','Lag5','Volume']].values
y_test = data.loc[data['Year']>2004]['direction2'].values

In [20]:
clf_logit.fit(X_train, y_train)

LogisticRegression(solver='newton-cg')

In [21]:
pred_test = clf_logit.predict(X_test)

In [22]:
confusion_matrix(y_test, pred_test).T

array([[48, 37],
       [93, 74]])

In [23]:
(48+74)/(48+74+37+93)

0.48412698412698413

## For some reason, let us try fitting the data with the most relevant features: Lag1 and Lag2.

In [24]:
X_train = data.loc[data['Year']<2005][['Lag1','Lag2']].values
y_train = data.loc[data['Year']<2005]['direction2'].values

X_test = data.loc[data['Year']>2004][['Lag1','Lag2']].values
y_test = data.loc[data['Year']>2004]['direction2'].values

In [25]:
clf_logit.fit(X_train, y_train)

LogisticRegression(solver='newton-cg')

In [26]:
pred_test = clf_logit.predict(X_test)

In [27]:
confusion_matrix(y_test, pred_test).T

array([[106,  76],
       [ 35,  35]])

In [28]:
(106+35)/(106+35+76+35)

0.5595238095238095

## LDA

In [29]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [30]:
lda = LinearDiscriminantAnalysis()
pred_test_lda = lda.fit(X_train, y_train).predict(X_test)

In [31]:
lda.priors_

array([0.50801603, 0.49198397])

In [32]:
lda.means_

array([[-0.03954635, -0.03132544],
       [ 0.04279022,  0.03389409]])

In [33]:
lda.coef_

array([[0.05544078, 0.0443452 ]])

In [34]:
confusion_matrix(y_test, pred_test_lda).T

array([[106,  76],
       [ 35,  35]])

In [35]:
(106+35)/(106+35+35+76)

0.5595238095238095

In [42]:
data.loc[data['Year']<2005].loc[data['Direction']=='Up']['Lag1'].mean()

-0.03954635108481257

In [43]:
data.loc[data['Year']<2005].loc[data['Direction']=='Up']['Lag2'].mean()

-0.031325443786982286

In [44]:
data.loc[data['Year']<2005].loc[data['Direction']=='Down']['Lag1'].mean()

0.04279022403258651

In [45]:
data.loc[data['Year']<2005].loc[data['Direction']=='Down']['Lag2'].mean()

0.0338940936863544

In [59]:
data.loc[data['Year']<2005].loc[data['Direction']=='Up']['Lag1'].var()

1.517005762175395

In [60]:
data.loc[data['Year']<2005].loc[data['Direction']=='Up']['Lag2'].var()

1.4902681527703063

In [61]:
data.loc[data['Year']<2005].loc[data['Direction']=='Down']['Lag1'].var()

1.5066227701899486

In [62]:
data.loc[data['Year']<2005].loc[data['Direction']=='Down']['Lag2'].var()

1.5355949765160637

## QDA

In [46]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [55]:
qda = QuadraticDiscriminantAnalysis(store_covariance=True)
pred_test_qda = qda.fit(X_train, y_train).predict(X_test)

In [56]:
qda.priors_

array([0.50801603, 0.49198397])

In [57]:
qda.means_

array([[-0.03954635, -0.03132544],
       [ 0.04279022,  0.03389409]])

In [58]:
qda.covariance_

[array([[ 1.51700576, -0.02787349],
        [-0.02787349,  1.49026815]]),
 array([[ 1.50662277, -0.03924806],
        [-0.03924806,  1.53559498]])]

In [50]:
confusion_matrix(y_test, pred_test_qda).T

array([[121,  81],
       [ 20,  30]])

In [41]:
(121+30)/(121+30+81+20)

0.5992063492063492

## k-nearest neighbor (kNN)

In [42]:
from sklearn import neighbors

In [43]:
knn = neighbors.KNeighborsClassifier(n_neighbors=1)
pred_knn_1 = knn.fit(X_train, y_train).predict(X_test)
print(confusion_matrix(y_test, pred_knn_1).T)

[[83 68]
 [58 43]]


In [44]:
(83+43)/(68+58+83+43)

0.5

In [45]:
knn = neighbors.KNeighborsClassifier(n_neighbors=3)
pred_knn_3 = knn.fit(X_train, y_train).predict(X_test)
print(confusion_matrix(y_test, pred_knn_3).T)

[[86 63]
 [55 48]]


In [46]:
(86+48)/(86+48+63+55)

0.5317460317460317

## What to explore next: potential class project on recent SP500 data
 - Collecting updated sp500 data in the same format: lag1-5, volume
 - what subsets of features are relevant for prediction? why? for instance, use bootstrap technique to see which features are relevant to direction.
 - besides the model above, you can use neural networks and other advanced models to predict
 - any other stock markets?